# Syriatel Customer Churn

### Authors
* Tony Bai
* Arthur Kim
* Eddie Lin
* Douglas Lu

## Overview

## Business Problem

## Data Understanding

## Import

In [1]:
#import relevant libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, OneHotEncoder

## Reviewing the Data File

In [2]:
#since the indiviudal notebook is in separate folder, had to move filepath one up and then import file

In [3]:
cd ..

/Users/arthur/Documents/Flatiron/phase_3/Project/SyriaTel_Customer_Churn


In [4]:
#read in the file
df = pd.read_csv('data/syriatel_customer_churn.csv')

In [5]:
#review the first five entires in the dataset
df.head()

,state,account length,area code,phone number,international plan,voice mail plan,number vmail messages,total day minutes,total day calls,total day charge,...,total eve calls,total eve charge,total night minutes,total night calls,total night charge,total intl minutes,total intl calls,total intl charge,customer service calls,churn
0,KS,128,415,382-4657,no,yes,25,265.1,110,45.07,...,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,371-7191,no,yes,26,161.6,123,27.47,...,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,358-1921,no,no,0,243.4,114,41.38,...,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,375-9999,yes,no,0,299.4,71,50.90,...,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,330-6626,yes,no,0,166.7,113,28.34,...,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3333 entries, 0 to 3332
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   state                   3333 non-null   object 
 1   account length          3333 non-null   int64  
 2   area code               3333 non-null   int64  
 3   phone number            3333 non-null   object 
 4   international plan      3333 non-null   object 
 5   voice mail plan         3333 non-null   object 
 6   number vmail messages   3333 non-null   int64  
 7   total day minutes       3333 non-null   float64
 8   total day calls         3333 non-null   int64  
 9   total day charge        3333 non-null   float64
 10  total eve minutes       3333 non-null   float64
 11  total eve calls         3333 non-null   int64  
 12  total eve charge        3333 non-null   float64
 13  total night minutes     3333 non-null   float64
 14  total night calls       3333 non-null   

## Cleaning the Data

After reviewing the dataset, we already see some interesting columns and also certain aspects that need to be modified. We see that the 'churn' column will be our target and needs to be converted into 0s and 1s. We also see that phone number is already split into the area code and the remaining seven numbers. For this project, we deemed that the area code may be more relevant for the project. We also noticed that the column names have spaces, which we will have to replace with underscores.

In [7]:
#convert the target 'churn' to 0s and 1s
df['churn'] = df['churn'].astype(int)

In [8]:
#replace the space in column names with underscores
df.columns = df.columns.str.replace(' ','_')

In [9]:
#drop the phone number column as it is not relevant for our model
df.drop('phone_number', axis=1, inplace=True)

In [10]:
df.head()

,state,account_length,area_code,international_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,customer_service_calls,churn
0,KS,128,415,no,yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,0
1,OH,107,415,no,yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,0
2,NJ,137,415,no,no,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,0
3,OH,84,408,yes,no,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,0
4,OK,75,415,yes,no,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,0


We noticed that some of the numeric columns were similar and can be summed to create new total columns. We created a total domestic minutes, calls, and charge columns that does not include international calls and separate total columns that do not include international calls. Charge per minute and the proportion of international calls may also play as factors for churn. We also created a customer call satisfaction column by taking the ratio of customer service calls to total calls, so that we can highlight customers who had a high volume of customer service calls relative to their total volume of calls. 

In [11]:
df['total_minutes_dom'] = df['total_day_minutes'] + df['total_eve_minutes'] + df['total_night_minutes']

In [12]:
df['total_calls_dom'] = df['total_day_calls'] + df['total_eve_calls'] + df['total_night_calls']

In [13]:
df['total_charge_dom'] = df['total_day_calls'] + df['total_eve_charge'] + df['total_night_charge']

In [14]:
df['total_minutes'] = df['total_day_minutes'] + df['total_eve_minutes'] + df['total_night_minutes'] + df['total_intl_minutes']

In [15]:
df['total_calls'] = df['total_day_calls'] + df['total_eve_calls'] + df['total_night_calls'] + df['total_intl_calls']

In [16]:
df['total_charge'] = df['total_day_calls'] + df['total_eve_charge'] + df['total_night_charge'] + df['total_intl_charge']

In [17]:
df["charge_per_min_dom"] = df['total_charge_dom']/df['total_calls_dom']

In [18]:
df["charge_per_min_int"] = df['total_intl_charge']/df['total_intl_calls']

In [20]:
df["%_int_min"]= df['total_intl_minutes']/ df['total_minutes']

In [22]:
df['call_satisfaction'] = df['customer_service_calls']/df['total_calls']

In [23]:
df.head()

,state,account_length,area_code,international_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,...,total_minutes_dom,total_calls_dom,total_charge_dom,total_minutes,total_calls,total_charge,charge_per_min_dom,charge_per_min_int,%_int_min,call_satisfaction
0,KS,128,415,no,yes,25,265.1,110,45.07,197.4,...,707.2,300,137.79,717.2,303,140.49,0.459300,0.900000,0.013943,0.003300
1,OH,107,415,no,yes,26,161.6,123,27.47,195.5,...,611.5,329,151.07,625.2,332,154.77,0.459179,1.233333,0.021913,0.003012
2,NJ,137,415,no,no,0,243.4,114,41.38,121.2,...,527.2,328,131.62,539.4,333,134.91,0.401280,0.658000,0.022618,0.000000
3,OH,84,408,yes,no,0,299.4,71,50.90,61.9,...,558.2,248,85.12,564.8,255,86.90,0.343226,0.254286,0.011686,0.007843
4,OK,75,415,yes,no,0,166.7,113,28.34,148.3,...,501.9,356,134.02,512.0,359,136.75,0.376461,0.910000,0.019727,0.008357


Thinking ahead of when we need to split out our categorical features, we decided to categorize the 'state' column into regions to create fewer categories rather can splitting out all 50 states and DC.

In [24]:
df['state'].nunique()

51

In [25]:
states = {
        'AK': 'West',
        'AL': 'South',
        'AR': 'South',
        'AZ': 'West',
        'CA': 'West',
        'CO': 'West',
        'CT': 'Northeast',
        'DC': 'south',
        'DE': 'South',
        'FL': 'South',
        'GA': 'South',
        'HI': 'West',
        'IA': 'Midwest',
        'ID': 'West',
        'IL': 'Midwest',
        'IN': 'Midwest',
        'KS': 'Midwest',
        'KY': 'South',
        'LA': 'South',
        'MA': 'Northeast',
        'MD': 'South',
        'ME': 'Northeast',
        'MI': 'Midwest',
        'MN': 'Midwest',
        'MO': 'Midwest',
        'MS': 'South',
        'MT': 'West',
        'NC': 'South',
        'ND': 'Midwest',
        'NE': 'Midwest',
        'NH': 'Northeast',
        'NJ': 'Northeast',
        'NM': 'West',
        'NV': 'West',
        'NY': 'Northeast',
        'OH': 'Midwest',
        'OK': 'South',
        'OR': 'West',
        'PA': 'Northeast',
        'RI': 'Northeast',
        'SC': 'South',
        'SD': 'Midwest',
        'TN': 'South',
        'TX': 'South',
        'UT': 'West',
        'VA': 'South',
        'VT': 'Northeast',
        'WA': 'West',
        'WI': 'Midwest',
        'WV': 'South',
        'WY': 'West'
}

In [26]:
df['region'] = df['state'].replace(states)

In [27]:
df.head()

,state,account_length,area_code,international_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,...,total_calls_dom,total_charge_dom,total_minutes,total_calls,total_charge,charge_per_min_dom,charge_per_min_int,%_int_min,call_satisfaction,region
0,KS,128,415,no,yes,25,265.1,110,45.07,197.4,...,300,137.79,717.2,303,140.49,0.459300,0.900000,0.013943,0.003300,Midwest
1,OH,107,415,no,yes,26,161.6,123,27.47,195.5,...,329,151.07,625.2,332,154.77,0.459179,1.233333,0.021913,0.003012,Midwest
2,NJ,137,415,no,no,0,243.4,114,41.38,121.2,...,328,131.62,539.4,333,134.91,0.401280,0.658000,0.022618,0.000000,Northeast
3,OH,84,408,yes,no,0,299.4,71,50.90,61.9,...,248,85.12,564.8,255,86.90,0.343226,0.254286,0.011686,0.007843,Midwest
4,OK,75,415,yes,no,0,166.7,113,28.34,148.3,...,356,134.02,512.0,359,136.75,0.376461,0.910000,0.019727,0.008357,South


## Exploratory Data Analysis (EDA)

In the exploratory data analysis (EDA) portion, we explored the target (churn) and the features included in the dataset

In [31]:
df_cont = df.select_dtypes(include=np.number).drop('area_code', axis=1)
df_cat = df[['churn', 'state', 'area_code', 'international_plan', 'voice_mail_plan', 'region']]

In [32]:
df_cont.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3333 entries, 0 to 3332
Data columns (total 26 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   account_length          3333 non-null   int64  
 1   number_vmail_messages   3333 non-null   int64  
 2   total_day_minutes       3333 non-null   float64
 3   total_day_calls         3333 non-null   int64  
 4   total_day_charge        3333 non-null   float64
 5   total_eve_minutes       3333 non-null   float64
 6   total_eve_calls         3333 non-null   int64  
 7   total_eve_charge        3333 non-null   float64
 8   total_night_minutes     3333 non-null   float64
 9   total_night_calls       3333 non-null   int64  
 10  total_night_charge      3333 non-null   float64
 11  total_intl_minutes      3333 non-null   float64
 12  total_intl_calls        3333 non-null   int64  
 13  total_intl_charge       3333 non-null   float64
 14  customer_service_calls  3333 non-null   

In [33]:
df_cat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3333 entries, 0 to 3332
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   churn               3333 non-null   int64 
 1   state               3333 non-null   object
 2   area_code           3333 non-null   int64 
 3   international_plan  3333 non-null   object
 4   voice_mail_plan     3333 non-null   object
 5   region              3333 non-null   object
dtypes: int64(2), object(4)
memory usage: 156.4+ KB
